<a href="https://colab.research.google.com/github/EmiLaPiola/NumLabIMC/blob/main/Copia_de_TP_autom%C3%A1tico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score

df = pd.read_csv('data.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

# Nueva sección

EMPEZAMOS CON EL SUPER MEGA TP DE LAS MAS LINDAS

Ejercicio 1
Separación de datos

Evaluar y justificar cómo separarán sus datos para desarrollo y para evaluación. ¿Qué consideraciones tuvieron en cuenta para realizar esta división?

Importante: en este punto no está permitido dividir la base de datos utilizando la función train_test_split de sklearn. Deben decidir e implementar la separación.

Primero separaremos las 500 instancias en un conjunto para utilizarlos como datos de desarrollo y otros para evaluación. Primero nos fijamos cuantas isntancias son positivas , y cuantas negativas . Nos fijamso en la variable "target" cuales son positivas y cuales no .

In [ ]:
conteo = df['target'].value_counts()

print(conteo)

target
0    353
1    147
Name: count, dtype: int64


El data set esta muy desbalanceado, hay mucha mas instancias con mal pronostico y solo un 29.4% de los datos tienen buen pronostico. Asi que para separar nuestros datos no lo haremos al azar. La separacion sera estratificada para que la proporicon de la clase minoritaria ( buen pronostico) sea preserve en ambos conjuntos. Utilizaremos 80 % de los datos para train y 20 % para control.

Tenemos 400 instancias en desarrolo set y 100 en el control set.  Tomamos el 20% de los datos positivos y 20% de datos negativos para ponerlos en el set de control .

Creamos dos data set distintos . Uno destinado para el desarrolo y otro para la evaluación.

In [ ]:

# Separamos positivos y negativos
positivos = df[df['target'] == True]
negativos = df[df['target'] == False]

# Seleccionamos al azar el 20% para control
control_positivos = positivos.sample(frac=0.2, random_state=42)
control_negativos = negativos.sample(frac=0.2, random_state=42)

# Concatenamos el set de control
NO_TOCAR_set = pd.concat([control_positivos, control_negativos])   # NO TOCAR

# El resto de los datos queda para desarrollo
desarrollo_set = df.drop(NO_TOCAR_set.index)

print(f"Desarrollo: {len(desarrollo_set)} instancias")
print(f"Control: {len(NO_TOCAR_set)} instancias")

Desarrollo: 400 instancias
Control: 100 instancias


Ejercicio 2

a) Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default.

Estimar la performance del modelo utilizando K-fold cross validation con K=5, con las métricas Accuracy, Area Under the Precision-Recall Curve (AUPRC), y Area Under the Receiver Operating Characteristic Curve (AUCROC).

✅ Ventajas de usar validación cruzada estratificada:

Mantiene la proporción de clases (True/False, o cualquiera que sea tu target) en cada fold.

Evita folds desbalanceados, que podrían llevar a:

Modelos entrenados solo con una clase (si el dataset es desbalanceado).

Métricas engañosas por una mala distribución de clases.

In [ ]:
# Hacemos 5 folds estratificados para que haya las mismas proprciones de clases minoritarias y mayoritarias en todos los folds.

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state = 22)

# Separamos de nuestro data set la columna "target"

desarrollo_set = desarrollo_set.reset_index(drop=True)
x_desarrollo = desarrollo_set.drop('target', axis=1)
y_desarrollo = desarrollo_set['target']

vector_accuracy_validacion= []  # aca vamos a guardar los resultados de acurracy para cada fold
vector_accuracy_train= []

vector_auprc_validacion = []
vector_auprc_train  = []

vector_auroc_validacion=[]
vector_auroc_train = []

# vector de prediciones
y_pred = np.empty(y_desarrollo.shape)
y_pred.fill(np.nan)

y_pred_prob = np.empty(y_desarrollo.shape)
y_pred_prob.fill(np.nan)


# generamos para cada fold una predicción
for train_index, test_index in folds.split(x_desarrollo,y_desarrollo):

        #saco el fold que no uso para entrenar
        kf_X_train, kf_X_test = x_desarrollo.iloc[train_index], x_desarrollo.iloc[test_index]
        kf_y_train, kf_y_test = y_desarrollo.iloc[train_index], y_desarrollo.iloc[test_index]

        # arbol fit con los datos de train
        arbol = DecisionTreeClassifier(max_depth=3)
        arbol.fit(kf_X_train, kf_y_train)
        y_pred_prob[test_index] = arbol.predict_proba(kf_X_test)[:, 1]

        # Hacemos las predicciones
        predictions = arbol.predict(kf_X_test)
        y_pred[test_index] = predictions

        # Métricas que nos piden comparar :  accuracy , AUPRC , AUCROC

        # Acuracy

        vector_accuracy_validacion.append(accuracy_score(kf_y_test, predictions))
        vector_accuracy_train.append(accuracy_score(kf_y_train, arbol.predict(kf_X_train)))



        # Curva AUPRC
        auprc = average_precision_score(kf_y_test, arbol.predict_proba(kf_X_test)[:, 1])
        vector_auprc_validacion.append(auprc)
        vector_auprc_train.append(average_precision_score(kf_y_train, arbol.predict_proba(kf_X_train)[:, 1]))

        # Curva AUROC
        auc_roc = roc_auc_score(kf_y_test, arbol.predict_proba(kf_X_test)[:, 1])
        vector_auroc_validacion.append(auc_roc)
        vector_auroc_train.append(roc_auc_score(kf_y_train, arbol.predict_proba(kf_X_train)[:, 1]))



# calculamos el acuracy
print(vector_accuracy_validacion)
print("El promedio de los accuracys por fold es:", round(np.mean(vector_accuracy_validacion),3))
print("El accuracy global es de : ", round(accuracy_score(y_desarrollo, y_pred),3))


# vamos con el auprc

print(vector_auprc_validacion)
print("El promedio de los Area Under the Precision-Recall Curve por fold es:", round(np.mean(vector_auprc_validacion),3))
print("El Area Under the Precision-Recall Curvec global es de : ", round(average_precision_score(y_desarrollo, y_pred),3))


# calculamos el auroc

print(vector_auroc_validacion)
print("El promedio de los Area Under the Receiver Operating Characteristic Curve por fold es:", round(np.mean(vector_auroc_validacion),3))
print("El Area Under the Receiver Operating Characteristic Curve global es de : ", round(roc_auc_score(y_desarrollo, y_pred),3))

print("AUPRC global:", round(average_precision_score(y_desarrollo, y_pred_prob), 3))
print("AUROC global:", round(roc_auc_score(y_desarrollo, y_pred_prob), 3))


[0.65, 0.75, 0.7125, 0.6875, 0.75]
El promedio de los accuracys por fold es: 0.71
El accuracy global es de :  0.71
[np.float64(0.4064932301024906), np.float64(0.5160103785103786), np.float64(0.42648256671089485), np.float64(0.36316136190278064), np.float64(0.514136641278585)]
El promedio de los Area Under the Precision-Recall Curve por fold es: 0.445
El Area Under the Precision-Recall Curvec global es de :  0.36
[np.float64(0.6164434523809524), np.float64(0.6863839285714286), np.float64(0.6748511904761905), np.float64(0.6372997711670481), np.float64(0.7784134248665141)]
El promedio de los Area Under the Receiver Operating Characteristic Curve por fold es: 0.679
El Area Under the Receiver Operating Characteristic Curve global es de :  0.59
AUPRC global: 0.431
AUROC global: 0.686


In [ ]:
tabla_resultados = pd.DataFrame({
    "Permutación": list(range(1, len(vector_accuracy_validacion)+1)),
    "Accuracy (training)": [round(x,3) for x in vector_accuracy_train],
    "Accuracy (validación)": [round(x,3) for x in vector_accuracy_validacion],
    "AUPRC (training)": [round(x,3) for x in vector_auprc_train],
    "AUPRC (validación)": [round(x,3) for x in vector_auprc_validacion],
    "AUC ROC (training)": [round(x,3) for x in vector_auroc_train],
    "AUC ROC (validación)": [round(x,3) for x in vector_auroc_validacion]
})

# Fila de promedios
promedios = {
    "Permutación": "Promedio",
    "Accuracy (training)": round(np.mean(vector_accuracy_train), 3),
    "Accuracy (validación)": round(np.mean(vector_accuracy_validacion), 3),
    "AUPRC (training)": round(np.mean(vector_auprc_train), 3),
    "AUPRC (validación)": round(np.mean(vector_auprc_validacion), 3),
    "AUC ROC (training)": round(np.mean(vector_auroc_train), 3),
    "AUC ROC (validación)": round(np.mean(vector_auroc_validacion), 3)
}

# Fila de métricas globales
globales = {
    "Permutación": "Global",
    "Accuracy (training)": "(NO)",
    "Accuracy (validación)": round(accuracy_score(y_desarrollo, y_pred), 3),
    "AUPRC (training)": "(NO)",
    "AUPRC (validación)": round(average_precision_score(y_desarrollo, y_pred), 3),
    "AUC ROC (training)": "(NO)",
    "AUC ROC (validación)": round(roc_auc_score(y_desarrollo, y_pred), 3)
}

tabla_resultados = pd.concat([tabla_resultados,
                              pd.DataFrame([promedios]),
                              pd.DataFrame([globales])],
                             ignore_index=True)

print(tabla_resultados)

  Permutación Accuracy (training)  Accuracy (validación) AUPRC (training)  \
0           1               0.856                  0.650             0.72   
1           2               0.853                  0.750            0.713   
2           3               0.853                  0.713            0.704   
3           4               0.847                  0.688            0.749   
4           5               0.844                  0.750            0.745   
5    Promedio               0.851                  0.710            0.726   
6      Global                (NO)                  0.710             (NO)   

   AUPRC (validación) AUC ROC (training)  AUC ROC (validación)  
0               0.406              0.874                 0.616  
1               0.516              0.842                 0.686  
2               0.426              0.829                 0.675  
3               0.363              0.855                 0.637  
4               0.514              0.848                 0

La clase positiva es la clase minoritaria . La AUPRC (Area Under Precision-Recall Curve) es muy sensible al desbalance de clases. Y como tenemos muy pocos positivos tiene sentido que nos de un promedio global muy bajo en la validacion de aupcr  # anotar cositas de las diapos de clase sobre estos temas xd .
Tambien tiene todo el sentido del mundo que eln los folds de entrenamiento nos de mucho mejor que en los sets de validacion ( trivial ) . Tanto roc como aupcr son sensibles a data set debalanceados como vimos en clase .

Tiene sentido que el promedio de igual al global en este caso particular porque los folds tienen todos el mismo tamaño... desarrollar :)

In [ ]:
## como la clase mayoritaria son los negativos invertimos las etiquetas
## pruebitas extras cambiando clases xd
##  cambiando las etiquetas de las predicciones observamos que las metricas son mejroes
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score
import numpy as np
import pandas as pd

# ---- Datos ----
desarrollo_set = desarrollo_set.reset_index(drop=True)
x_desarrollo = desarrollo_set.drop('target', axis=1)
y_desarrollo = desarrollo_set['target']

# Invertimos la clase: la clase mayoritaria = 1, minoritaria = 0
y_desarrollo_invertida = 1 - y_desarrollo

# ---- Estratificación ----
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=22)

# ---- Vectores para métricas ----
vector_accuracy_validacion = []
vector_accuracy_train = []

vector_auprc_validacion = []
vector_auprc_train = []

vector_auroc_validacion = []
vector_auroc_train = []

# ---- Predicciones globales ----
y_pred = np.empty(y_desarrollo.shape)
y_pred.fill(np.nan)

y_pred_prob = np.empty(y_desarrollo.shape)
y_pred_prob.fill(np.nan)

# ---- Loop por fold ----
for train_index, test_index in folds.split(x_desarrollo, y_desarrollo_invertida):
    kf_X_train, kf_X_test = x_desarrollo.iloc[train_index], x_desarrollo.iloc[test_index]
    kf_y_train, kf_y_test = y_desarrollo_invertida.iloc[train_index], y_desarrollo_invertida.iloc[test_index]

    # Entrenar modelo
    arbol = DecisionTreeClassifier(max_depth=3)
    arbol.fit(kf_X_train, kf_y_train)

    # Predicciones discretas y probabilidades
    y_pred_fold = arbol.predict(kf_X_test)
    y_pred_prob_fold = arbol.predict_proba(kf_X_test)[:,1]

    # Guardar global
    y_pred[test_index] = y_pred_fold
    y_pred_prob[test_index] = y_pred_prob_fold

    # Métricas por fold
    vector_accuracy_validacion.append(accuracy_score(kf_y_test, y_pred_fold))
    vector_accuracy_train.append(accuracy_score(kf_y_train, arbol.predict(kf_X_train)))

    vector_auprc_validacion.append(average_precision_score(kf_y_test, y_pred_prob_fold))
    vector_auprc_train.append(average_precision_score(kf_y_train, arbol.predict_proba(kf_X_train)[:,1]))

    vector_auroc_validacion.append(roc_auc_score(kf_y_test, y_pred_prob_fold))
    vector_auroc_train.append(roc_auc_score(kf_y_train, arbol.predict_proba(kf_X_train)[:,1]))

# ---- Promedios por fold ----
promedio_accuracy_train = np.mean(vector_accuracy_train)
promedio_accuracy_val = np.mean(vector_accuracy_validacion)

promedio_auprc_train = np.mean(vector_auprc_train)
promedio_auprc_val = np.mean(vector_auprc_validacion)

promedio_auroc_train = np.mean(vector_auroc_train)
promedio_auroc_val = np.mean(vector_auroc_validacion)

# ---- Score global ----
accuracy_global = accuracy_score(y_desarrollo_invertida, y_pred)
auprc_global = average_precision_score(y_desarrollo_invertida, y_pred_prob)
auroc_global = roc_auc_score(y_desarrollo_invertida, y_pred_prob)

# ---- Crear tabla ----
tabla = pd.DataFrame({
    'Fold': list(range(1,6)),
    'Accuracy_train': vector_accuracy_train,
    'Accuracy_validacion': vector_accuracy_validacion,
    'AUPRC_train': vector_auprc_train,
    'AUPRC_validacion': vector_auprc_validacion,
    'AUROC_train': vector_auroc_train,
    'AUROC_validacion': vector_auroc_validacion
})

# Agregar fila de promedios
tabla_promedios = pd.DataFrame({
    'Fold': ['Promedio'],
    'Accuracy_train': [promedio_accuracy_train],
    'Accuracy_validacion': [promedio_accuracy_val],
    'AUPRC_train': [promedio_auprc_train],
    'AUPRC_validacion': [promedio_auprc_val],
    'AUROC_train': [promedio_auroc_train],
    'AUROC_validacion': [promedio_auroc_val]
})

# Agregar fila de global
tabla_global = pd.DataFrame({
    'Fold': ['Global'],
    'Accuracy_train': [np.nan],
    'Accuracy_validacion': [accuracy_global],
    'AUPRC_train': [np.nan],
    'AUPRC_validacion': [auprc_global],
    'AUROC_train': [np.nan],
    'AUROC_validacion': [auroc_global]
})

# Concatenar todas las filas
tabla_final = pd.concat([tabla, tabla_promedios, tabla_global], ignore_index=True)
print(tabla_final)








       Fold  Accuracy_train  Accuracy_validacion  AUPRC_train  \
0         1        0.856250               0.6375     0.916560   
1         2        0.853125               0.7500     0.890516   
2         3        0.853125               0.7250     0.880781   
3         4        0.846875               0.6875     0.898649   
4         5        0.843750               0.7500     0.891900   
5  Promedio        0.850625               0.7100     0.895681   
6    Global             NaN               0.7100          NaN   

   AUPRC_validacion  AUROC_train  AUROC_validacion  
0          0.758307     0.873917          0.587426  
1          0.796136     0.841838          0.686384  
2          0.816861     0.828775          0.692708  
3          0.802280     0.855251          0.637300  
4          0.876708     0.847673          0.778413  
5          0.810058     0.849491          0.676446  
6          0.806219          NaN          0.681858  


1.3 Probar distintos parámetros y hacer tabla

In [ ]:
# Probamos disitnos parametros para nuestro arbol de decision

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state = 22)

alturas = [1,2,3,5,10,None]
criterios = ['gini','entropy']
accuracy_validacion = {}
accuracy_train = {}

for a in alturas:
    for c in criterios:
        vector_accuracy_validacion= []  # aca vamos a guardar los resultados de acurracy para cada fold de la validacion
        vector_accuracy_train= [] # y del training

        # generamos para cada fold una predicción
        for train_index, test_index in folds.split(x_desarrollo,y_desarrollo):

          #saco el fold que no uso para entrenar
          kf_X_train, kf_X_test = x_desarrollo.iloc[train_index], x_desarrollo.iloc[test_index]
          kf_y_train, kf_y_test = y_desarrollo.iloc[train_index], y_desarrollo.iloc[test_index]

          # arbol fit con los datos de train
          arbol = DecisionTreeClassifier(max_depth=a, criterion=c)
          arbol.fit(kf_X_train, kf_y_train)

          y_pred_val = arbol.predict(kf_X_test)
          acc_val = accuracy_score(kf_y_test, y_pred_val)
          vector_accuracy_validacion.append(acc_val)

          y_pred_train = arbol.predict(kf_X_train)
          acc_train = accuracy_score(kf_y_train, y_pred_train)
          vector_accuracy_train.append(acc_train)

        # Guardamos el promedio de accuracy de los folds
        accuracy_validacion[(a, c)] = np.mean(vector_accuracy_validacion)
        accuracy_train[(a, c)] = np.mean(vector_accuracy_train)

print(accuracy_validacion)
print(accuracy_train)
# El mejor es gini con altura 5

import pandas as pd

# Lista para guardar filas de la tabla
tabla_resultados = []

# Recorremos las claves del diccionario (pares de (altura, criterio))
for clave in accuracy_train.keys():
    altura, criterio = clave
    acc_train = accuracy_train[clave]
    acc_val = accuracy_validacion[clave]

    # Agregamos una fila a la tabla
    tabla_resultados.append({
        'Altura': altura,
        'Criterio': criterio,
        'Accuracy Train': acc_train,
        'Accuracy Validación': acc_val
    })

# Creamos el DataFrame
df_resultados = pd.DataFrame(tabla_resultados)

# Mostramos la tabla ordenada por Altura y Criterio
df_resultados = df_resultados.sort_values(by=['Altura', 'Criterio']).reset_index(drop=True)
print(df_resultados)


{(1, 'gini'): np.float64(0.6775), (1, 'entropy'): np.float64(0.6775), (2, 'gini'): np.float64(0.7224999999999999), (2, 'entropy'): np.float64(0.735), (3, 'gini'): np.float64(0.7075), (3, 'entropy'): np.float64(0.7150000000000001), (5, 'gini'): np.float64(0.7175), (5, 'entropy'): np.float64(0.6975), (10, 'gini'): np.float64(0.6849999999999999), (10, 'entropy'): np.float64(0.6525000000000001), (None, 'gini'): np.float64(0.7), (None, 'entropy'): np.float64(0.6725000000000001)}
{(1, 'gini'): np.float64(0.7093750000000001), (1, 'entropy'): np.float64(0.7093750000000001), (2, 'gini'): np.float64(0.784375), (2, 'entropy'): np.float64(0.779375), (3, 'gini'): np.float64(0.850625), (3, 'entropy'): np.float64(0.825), (5, 'gini'): np.float64(0.944375), (5, 'entropy'): np.float64(0.9356250000000002), (10, 'gini'): np.float64(0.9956250000000001), (10, 'entropy'): np.float64(1.0), (None, 'gini'): np.float64(1.0), (None, 'entropy'): np.float64(1.0)}
    Altura Criterio  Accuracy Train  Accuracy Valida

Claramente a medida de que aumentamos el max_depth del arbol, vemos que la accuracy en los datos de train tiende a 100%, lo que tiene sentido, ya que calsifica bien todos estos datos (los memoriza). A su vez, para los datos de test. resulta peor. observamos overfitting.

Estuvimos probando con otros valores y llegamos a la conclusion que la mejor altura es 2, ya que no solo, comparado a la performarnce en el test con otra alturas, es mejor, sino que tambien la diferencia con los datos de training es menor, osea que esta generalizando bastante bien, y no memoriza solo los datos de entrenamiento.


Ejercicio 3
Importante: de acá en más sólamente utilizaremos el score promedio cuando hagamos K-fold cross-validation.

Para el arbol de decision elegimos jugar con los siguientes hiperparametros:


*   Max_depth (para regular el tamaño del arbol)
*   class_weight (para tratar el desbalanceo de clases que tenemos)
*   criterion
*   min_samples_split (para definir cuantos hijos vamos a tener por rama como maximo)

Por otro lado, para knn elegimos:
* La cantidad de vecinos (k)
* weights (darle mas peso a los que estan mas cercanos y menos a los mas alejados)
* metrics (el tipo de distancia que usamos)

Finalmente para SVM:
* C (penalizacion por clasificacion equivocada)
* kernel (hay distintos tipos)
* class_weight (al ser desbalanceada nos conviene jugar con esto)
